In [ ]:
import dataLoading
import wolframPreprocessing
import myWMEncoder
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from unixcoder import UniXcoder

In [2]:
loader = dataLoading.DataLoaderExporter()
preprocessor = wolframPreprocessing.WolframPreprocessor()

In [5]:
loader.make_json_data()

In [ ]:
data = loader.load_json_data()
data

In [ ]:
data["transformed_code"] = data["code"].apply(lambda x: preprocessor.preprocessing_composition_naive(x, tokenize=True))
data.drop(data[data["transformed_code"].apply(lambda x: len(x) == 0)].index, inplace=True)
data["tr_code_non_tokenized"] = data["code"].apply(lambda x: preprocessor.preprocessing_composition_naive(x, tokenize=False))
data.reset_index(drop=True, inplace=True)

In [8]:
data = data[data["task"] == data["task"].unique()[0]]

In [9]:
encoder = myWMEncoder.CustomEncoder(50)
encoder.fit(data["transformed_code"].values)
encoder_size = encoder.vec_size

In [10]:
class WindowDataset(Dataset):
    def __init__(self, data, encoder, window_size=5):
        self.data = data
        self.encoder = encoder
        self.encoder.fit(data)
        self.window_size = window_size
        self.windows = None
        self.labels = np.array([])
        self.start = True

        for string in self.data:
            tmp = sliding_window_view(string, window_size + 1)
            for win in tmp:
                encoded_win = self.encoder.transform(win)
                x = np.array([encoded_win[:-1]])
                label = np.array([np.argmax(encoded_win[-1][:-2])])
                if self.start:
                    self.windows = x
                    self.start = False
                else:
                    self.windows = np.concatenate((self.windows, x))
                self.labels = np.concatenate((self.labels, label))

        self.labels = torch.LongTensor(self.labels)
        self.windows = torch.FloatTensor(self.windows)

    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        return self.windows[idx], self.labels[idx]

In [11]:
class WindowDatasetOpt(Dataset):
    def __init__(self, data, encoder, window_size=5):
        self.data = data
        self.encoder = encoder
        self.encoder.fit(data)
        self.window_size = window_size
        self.windows = None
        self.labels = np.array([])
        self.start = True

        for code in self.data:
            transformed_code = self.encoder.transform(code)
            sliding_windows = sliding_window_view(transformed_code, (self.window_size + 1, self.encoder.vec_size)).reshape(-1, self.window_size + 1, self.encoder.vec_size)
            x = sliding_windows[:, :-1]
            labels = np.argmax(sliding_windows[:, -1,: -2], axis=1)
            if self.start:
                self.windows = x
                self.labels = labels
                self.start = False
            else:
                self.windows = np.concatenate((self.windows, x))
                self.labels = np.concatenate((self.labels, labels))
        self.labels = torch.LongTensor(self.labels)
        self.windows = torch.FloatTensor(self.windows)

    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        return self.windows[idx], self.labels[idx]

In [12]:
enc = myWMEncoder.CustomEncoder(50)
new_dataset = WindowDatasetOpt(data["transformed_code"].values, enc, 10)

In [14]:
trainloader = DataLoader(new_dataset, batch_size=64, shuffle=True)

In [15]:
class LSTMWindow(nn.Module):
    def __init__(self, hidden_size, vec_size):
        super(LSTMWindow, self).__init__()
        
        self.hidden_size = hidden_size
        self.vec_size = vec_size
        self.n_classes = vec_size - 2
        self.lstm = nn.LSTM(self.vec_size,
                        self.hidden_size,
                        num_layers=1,
                        batch_first=True,
                        bidirectional=False
                        )
        self.lin_layer = nn.Linear(self.hidden_size, self.n_classes)
        
    def forward(self, inputs):
        h_0 = torch.zeros(1, inputs.size(0), self.hidden_size)
        c_0 = torch.zeros(1, inputs.size(0), self.hidden_size)
        output_features, (h_out, _) = self.lstm(inputs, (h_0, c_0))  
        self.h_out = h_out.view(-1, self.hidden_size)  
        return self.lin_layer(self.h_out)
    
    def emb(self, inputs):
        self.forward(inputs)
        return self.h_out

In [16]:
torch.manual_seed(0)
model = LSTMWindow(300, new_dataset.encoder.vec_size)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [17]:
from tqdm import tqdm
from tqdm import trange

In [18]:
for epoch in trange(50, desc="Learning:", unit="carrots"):
    i = 1
    mean_sum = 0
    for id, (texts, targets) in enumerate(trainloader):
    
        optimizer.zero_grad()
        pred = model(texts)
        loss = criterion(pred, targets)
        mean_sum += loss.item()
        i += 1
        loss.backward()
        optimizer.step()
    print(f"Epoch[{epoch + 1}/100], mean_epoch_loss: {mean_sum/i}")

Learning::   2%|▏         | 1/50 [00:04<03:55,  4.80s/carrots]

Epoch[1/100], mean_epoch_loss: 2.1130449447265036


Learning::   4%|▍         | 2/50 [00:09<03:49,  4.78s/carrots]

Epoch[2/100], mean_epoch_loss: 1.2032436517568734


Learning::   6%|▌         | 3/50 [00:14<03:43,  4.75s/carrots]

...

Epoch[48/100], mean_epoch_loss: 0.24834877635423955


Learning::  98%|█████████▊| 49/50 [03:50<00:04,  4.62s/carrots]

Epoch[49/100], mean_epoch_loss: 0.21012466879991384


Learning:: 100%|██████████| 50/50 [03:55<00:00,  4.71s/carrots]

Epoch[50/100], mean_epoch_loss: 0.16424053297019922


In [ ]:
enc_need = new_dataset.encoder

In [20]:
works = [model.emb(torch.FloatTensor(np.array([enc_need.transform(tokens)])))[0].tolist() for tokens in data["transformed_code"].values]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from Levenshtein import ratio

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_unix = UniXcoder("microsoft/unixcoder-base")
model_unix.to(device)

In [ ]:
lstm_embs, unixcoder_embs = {}, {}
for name, code in enumerate(data[['name', 'task']].values):
    tokens_ids = model.tokenize([code],max_length=512,mode="<encoder-only>")
    source_ids = torch.tensor(tokens_ids).to(device)
    _, unix = model(source_ids)

    lstm = model.emb([code])
    lstm_embs[name] = lstm
    unixcoder_embs[name] = unix


In [ ]:
def compute_unix_embs(ind_pairs):
    unix_values = []
    for pair in ind_pairs:
        sim_unix = torch.einsum("ac,bc->ab",
                                unixcoder_embs[pair[0]],
                                unixcoder_embs[pair[1]])[0][0].item()
        unix_values.append(sim_unix)
    return unix_values

def compute_lstm_embs(ind_pairs):
    lstm_values = []
    for pair in ind_pairs:
        sim_lstm = cosine_similarity(lstm_embs[pair[0]], lstm_embs[pair[1]])[0][0]
        lstm_values.append(sim_lstm)
    return lstm_values


In [ ]:
def compute_plags_alg1(clust, vertecies, models_weights, threshold):
    total_info = []
    for v0_ind in range(len(vertecies)):
        for v1_ind in range(len(vertecies)):
            if v0_ind < v1_ind:
                v0, v1 = vertecies[[v0_ind, v1_ind]]
                works_v0 = clust[clust["name"] == v0][["file", "index"]].values
                works_v1 = clust[clust["name"] == v1][["file", "index"]].values
                plags = []
                for v0_file, v0_edge in works_v0:
                    s0 = clust[clust["index"] == v0_edge]["tr_code_non_tokenized"].values[0]
                    for v1_file, v1_edge in works_v1:
                        s1 = clust[clust["index"] == v1_edge]["tr_code_non_tokenized"].values[0]
                        sim_leven = Levenshtein.ratio(s0, s1)
                        sim_unix = torch.einsum("ac,bc->ab",unixcoder_embs[v0_edge],unixcoder_embs[v1_edge])[0][0].item()
                        sim_lstm = cosine_similarity(lstm_embs[v0_edge], lstm_embs[v1_edge])[0][0]
                        total_sim = models_weights @ [sim_leven, sim_lstm, sim_unix]
                        if total_sim > threshold:
                            plags.append([v0_file, v1_file, (sim_leven, sim_lstm, sim_unix, total_sim)])
                if len(plags) > 0:
                    total_info.append([v0, v1, plags, len(plags),"label", "label"])
    return total_info    


In [ ]:
def compute_plags_alg2(clust, vertecies, models_weights, threshold_lvn_upper, threshold_lvn_lower, threshold_nn):
    total_info = []
    for v0_ind in range(len(vertecies)):
        for v1_ind in range(len(vertecies)):
            if v0_ind < v1_ind:
                v0, v1 = vertecies[[v0_ind, v1_ind]]
                works_v0 = clust[clust["name"] == v0][["file", "index"]].values
                works_v1 = clust[clust["name"] == v1][["file", "index"]].values
                plags = []
                for v0_file, v0_edge in works_v0:
                    s0 = clust[clust["index"] == v0_edge]["tr_code_non_tokenized"].values[0]
                    for v1_file, v1_edge in works_v1:
                        s1 = clust[clust["index"] == v1_edge]["tr_code_non_tokenized"].values[0]
                        sim_leven = Levenshtein.ratio(s0, s1)
                        sim_unix = torch.einsum("ac,bc->ab",unixcoder_embs[v0_edge],unixcoder_embs[v1_edge])[0][0].item()
                        sim_lstm = cosine_similarity(lstm_embs[v0_edge], lstm_embs[v1_edge])[0][0]

                        if sim_leven >= threshold_lvn_upper:
                            plags.append([v0_file, v1_file, (sim_leven, 0, 0, sim_leven)])
                        elif threshold_lvn_lower <= sim_leven < threshold_lvn_upper:
                            nn_sim = models_weights @ [sim_lstm, sim_unix]
                            if nn_sim >= threshold_nn:
                                plags.append([v0_file, v1_file, (sim_leven, 0, sim_unix, 0)])

                if len(plags) > 0:
                    total_info.append([v0, v1, plags, len(plags),"label", "label"])
